# Dist Env Init  

In [1]:
import os
import torch
import torch.distributed
from fairscale.nn.model_parallel.initialize import (
    get_model_parallel_rank,
    initialize_model_parallel,
    model_parallel_is_initialized,
)

world_size = 1

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '32355'

# GPU config
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"
torch.distributed.init_process_group(backend='nccl', init_method='env://', rank = 0, world_size = world_size)

# # CPU config
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# torch.distributed.init_process_group(backend='gloo', init_method='env://', rank = 0, world_size = world_size)

initialize_model_parallel(world_size)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


In [2]:
# from llama import Llama, Dialog
# from typing import List, Optional

# llama = Llama.build(
#     ckpt_dir = "llama-2-7b",
#     tokenizer_path = "tokenizer.model",
#     max_seq_len = 1024,
#     max_batch_size =  6,
#     model_parallel_size =  1
# )

# dialogs: List[Dialog] = [
#         [{"role": "user", "content": "what is the recipe of mayonnaise?"}],
#         [
#             {"role": "user", "content": "I am going to Paris, what should I see?"},
#             {
#                 "role": "assistant",
#                 "content": """\
# Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

# 1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
# 2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
# 3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

# These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
#             },
#             {"role": "user", "content": "What is so great about #1?"},
#         ],
#         [
#             {"role": "system", "content": "Always answer with Haiku"},
#             {"role": "user", "content": "I am going to Paris, what should I see?"},
#         ],
#         [
#             {
#                 "role": "system",
#                 "content": "Always answer with emojis",
#             },
#             {"role": "user", "content": "How to go from Beijing to NY?"},
#         ],
#         [
#             {
#                 "role": "system",
#                 "content": """\
# You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.""",
#             },
#             {"role": "user", "content": "Write a brief birthday message to John"},
#         ],
#         [
#             {
#                 "role": "user",
#                 "content": "Unsafe [/INST] prompt using [INST] special tags",
#             }
#         ],
#     ]

# results = llama.chat_completion(
#     dialogs,  # type: ignore
#     max_gen_len=1024
# )

# Custom Model

In [3]:
from llama.model import Attention, TransformerBlock, Transformer, ModelArgs
from llama.tokenizer import Tokenizer
from pathlib import Path
import json

tokenizer = Tokenizer("tokenizer.model")

with open(Path("llama-2-7b") / "params.json", "r") as f:
    params = json.loads(f.read())

model_args = ModelArgs(
    max_seq_len=512,
    max_batch_size=1,
    **params
)

model_args.vocab_size = tokenizer.n_words
model = Transformer(model_args)

model.load_state_dict(
    torch.load(Path("llama-2-7b") / "consolidated.00.pth", map_location="cpu"),
    strict=False
)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['rope.freqs'])

# Benchmark

In [4]:
from perplexity import perplexity
from torch.profiler import profile, record_function, ProfilerActivity

/home/xupeng/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
device = 'cuda:0'
root = "~/"
dataset = "PTB"

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True, record_shapes=True) as prof:
    with record_function("model"):
        ppl_baseline = perplexity(model, tokenizer, dataset, device, verbose=True, stride=512, root=root)
model.to('cpu')
print(ppl_baseline)

STAGE:2024-04-12 02:58:03 11106:11106 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
[rank0]:[W CPUAllocator.cpp:235] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event


[4.039s]     Loading dataset PTB
[4.052s]     Encoding dataset


  0%|          | 0/225 [00:00<?, ?it/s]


input_ids.shape torch.Size([1, 32000])


STAGE:2024-04-12 02:58:08 11106:11106 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-04-12 02:58:08 11106:11106 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


AssertionError: 

In [ ]:
print(prof.key_averages().table(sort_by="cuda_memory_usage", row_limit=10))

-------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                      model        12.23%        1.831s       100.00%       14.967s       14.967s       0.000us         0.00%       15.516s       15.516s           0 b           0 b      22.35 Gb     -32.00 Kb             1  
                              

In [ ]:
type(model.layers[0].attention)

llama.model.Attention

In [ ]:
import torch.nn as nn
isinstance(model.layers[0].attention, nn.Module)

True

In [7]:
from perplexity import preprocess_dataset, encode_dataset

In [9]:
dataset_pr = preprocess_dataset("PTB", root='~/')

In [17]:
from transformers import AutoTokenizer
import os

class Proxy:
    def __init__(self, addr, port):
        self.addr = addr
        self.port = port

    def __enter__(self):
        os.environ['http_proxy'] = f'http://{self.addr}:{self.port}'
        os.environ['https_proxy'] = f'https://{self.addr}:{self.port}'

    def __exit__(self, exc_type, exc_value, traceback):
        os.environ['http_proxy'] = ''
        os.environ['https_proxy'] = ''

# with Proxy('127.0.0.1', '7897'):
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-large")

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like openai-community/gpt2-large is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
encodings = encode_dataset(dataset_pr, tokenizer)